# Marangoni boat project code

Welcome to this notebook with code to be used for the data analysis of the boats :) This notebook contains code adapted from original code written by Jackson Wilt and Nico Schramma.
<br />
This notebook contains all data you can use for analysis of the boats, with some comments on how it works. Feel free to copy the notebook and make changes or adaptations however you see fit.

In [1]:

# Importing relevant packages
import cv2
import numpy as np
import pims
import trackpy as tp
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from copy import copy

## Loading in video
In the cell below you can put the name of the video you want to be analysed. Make sure you put the video in the same folder as where the file with this code is put.

In [2]:
fileName = r'Tracking_videos\7-4-2025\nozzle_0.1mmTOL.mp4'

In [3]:
video = pims.Video(fileName) # Loading in video and converting them to individual frames
duration = video.duration # Duration of video in seconds
frameRate = video.frame_rate # Framerate of video

print(f'duration:{round(duration,3)}, framerate: {round(frameRate,3)}, # of frames: {round(duration * frameRate)}')

duration:38.669, framerate: 29.998, # of frames: 1160


Use the snippet of code underneath if you want to only analyse a selection of frames. When working with large videos it is advisable to first test your code on a small selection of frames. This way you can quickly see if everything works as expected!

In [63]:
startframe = 180
endframe = 1160
frames = video[startframe:endframe]
print('Amount of frames:', len(frames))

Amount of frames: 979


In [71]:
def boatCentre(frames):
    """
    Args:
        frames to be analysed

    Returns:
       positions [np.array]: list of frames with only the positions of the tracking dots highlited.
       blurs [np.array]: list of frames with the results of the blurring and clipping shown.
       newset [np.array]: list of original frames with the tracked dots highlited with circles.
       circstore [list]: list with lists of tuples containing the information (pos & radius) of each circle per frame.
       indecesused [list]: list of the indeces corresponding to the frames that were processed from the frames input.
    """
    circstore = [] # list of lists with tuples containing the info of the circles per frame
    shorten = 1 #10
    reducefps = 3
    colorthreshold = 0.35 # threshold value for when to count the pixel as a tracking cap 0 means everything counts
    positions, blurs, newset = [], [], [] #lists to be filled with their respective frame types
    indecesused = []

    for i in range(0,int(len(frames)/shorten),reducefps):
        indecesused.append(i)
        cimg= copy(frames[i]) # makes a copy of the original frame as to not edit over the original

        r,gr,b = cv2.split(frames[i]) # split the color image into different channels    
        blurthis = (abs((r/255)-(gr/255))+abs((gr/255)-(b/255))+abs((r/255)-(b/255))) # calculate the color difference for each pixel from 0-1 (this highlights pixels with a lot of a single color)

        blurthis[blurthis>=colorthreshold] = 255 # set color difference values above a threshold to max intensity
        blurthis[blurthis<colorthreshold] = 0 # set color difference values below the same threshold to zero intenisty
        blur = cv2.medianBlur(np.uint8(blurthis), ksize=7) # blur to help houghcircles. If all went well, only the colored dots on the cheerioboat remain.

        circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, dp=1.5, minDist=35, param1=60, param2=10, minRadius=2, maxRadius=20) # actually find the circles in the edited image 
        circles = np.uint16(np.around(circles)) # round the found circle positions and radii
        
        circstore.append(circles) # save the positions and radii of the circles
        # blur = cv2.cvtColor(blur,cv2.COLOR_GRAY2RGB)

        background = np.zeros_like(blur, np.uint8) # create a black greyscale background 
    
        for j in circles[0,:]:
            # draw the centers of the circles
            cv2.circle(background,(j[0],j[1]),3,(255,0,0),-1) 
            cv2.circle(blur,(j[0],j[1]),3,255, -1) 
            cv2.circle(cimg,(j[0],j[1]),3,(255,0,0),-1) 
            
            # draw the edge of the circles
            # cv2.circle(background,(i[0],i[1]),i[2],(0,255,0),1)
            cv2.circle(blur,(j[0],j[1]),j[2],(0,255,0),1)
            cv2.circle(cimg,(j[0],j[1]),j[2],(0,255,0),1)
        
        positions.append(background)
        blurs.append(blur)
        newset.append(cimg)

        print('finished frame:', i)
    
    return np.array(positions), np.array(blurs), np.array(newset), circstore, indecesused

## Editing all frames

In [72]:
positions, _, newset, _, indecesused,= boatCentre(frames) # returns frames with dots placed in the centre of mass of the boats for all frames

# print('aantal cirkels:', int(len(np.ndarray.flatten(np.array(circstore)))/3))

finished frame: 0
finished frame: 3
finished frame: 6
finished frame: 9
finished frame: 12
finished frame: 15
finished frame: 18
finished frame: 21
finished frame: 24
finished frame: 27
finished frame: 30
finished frame: 33
finished frame: 36
finished frame: 39
finished frame: 42
finished frame: 45
finished frame: 48
finished frame: 51
finished frame: 54
finished frame: 57
finished frame: 60
finished frame: 63
finished frame: 66
finished frame: 69
finished frame: 72
finished frame: 75
finished frame: 78
finished frame: 81
finished frame: 84
finished frame: 87
finished frame: 90
finished frame: 93
finished frame: 96
finished frame: 99
finished frame: 102
finished frame: 105
finished frame: 108
finished frame: 111
finished frame: 114
finished frame: 117
finished frame: 120
finished frame: 123
finished frame: 126
finished frame: 129
finished frame: 132
finished frame: 135
finished frame: 138
finished frame: 141
finished frame: 144
finished frame: 147
finished frame: 150
finished frame: 15

In [73]:
%matplotlib tk
fig, axs = plt.subplots(1,2, figsize=(10,6))

axs[0].imshow(newset[4])
axs[0].set_title('Frame')
axs[1].imshow(positions[5])
axs[1].set_title('Centre point of boats')

plt.show()

The cell below will search for all the dots we put in the frames and save their positions.

In [74]:
f = tp.batch(positions, diameter=5, minmass=20, processes='auto', invert=False)#37

Frame 326: 3 features


We've now got locations for all our dots! But to get information on the movement of the boats it is important that we can label boats. The code snippet below will compare all our dots frame by frame and detect which ones are which boat.

In [75]:
search_range = 140 
memory       = 20

t = tp.link(f, search_range=search_range, memory=memory)# WAS 90
# t1 = tp.filter_stubs(t, 50)
# t1['t']=t1['frames']

Frame 326: 3 trajectories present.


In [76]:
t

,y,x,mass,size,ecc,signal,raw_mass,ep,frame,particle
0,884.856934,392.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,0,0
1,907.856934,424.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,0,1
2,1411.856934,1426.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,0,2
3,883.856934,374.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,1,0
4,908.856934,404.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...
974,1408.856934,1421.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,325,2
973,668.856934,587.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,325,1
977,1408.856934,1421.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,326,2
976,667.856934,584.856934,398.560505,1.395794,0.217477,49.625454,2805.0,0.0,326,1


We've now got the trajectory of our boats! :D

In [84]:
plt.axes().set_aspect('equal')
if t['particle'].max() != 0:
    for i in np.arange(0, t['particle'].max()):
        tt = t.loc[t['particle']==i]
        plt.plot(tt['x'], tt['y'], label = i)
else:
    tt = t.loc[t['particle']==0]
    plt.scatter(tt['x'], tt['y'], c = tt['frame'], s = 0.7)
    plt.plot(tt['x'], tt['y'], color = 'black', zorder = -10, linewidth = 0.5)
    
plt.imshow(frames[0], zorder = -20)
plt.legend()

In [85]:
if t['particle'].max() != 0:
    for i in np.arange(0, t['particle'].max()):
        tt = t.loc[t['particle']==i]
        plt.plot(tt['frame'], np.sqrt(tt['y']**2 + tt['x']**2), label = i)
else:
    plt.plot(tt['frame'], np.sqrt(tt['y']**2 + tt['x']**2), label = 0)
plt.legend()

## Average speed

In [88]:
displacement = []
averageSpeed = []
for i in np.arange(0, t['particle'].max()):
    tt = t.loc[t['particle'] == i]
    tt = tt.reset_index()
    S = 0 
    for j in range(1, len(tt['x'])):
        ds = np.sqrt((tt['x'].iloc[j] - tt['x'].iloc[j - 1])**2 + (tt['y'].iloc[j] - tt['y'].iloc[j - 1])**2)
        S = S + ds
    displacement.append(S)
    averageSpeed.append(S/duration)

data = {
    'particle':  np.arange(t['particle'].max()),
    'displacement' : displacement,
    'averageSpeed' : averageSpeed
}
df = pd.DataFrame(data)
display(df)

,particle,displacement,averageSpeed
0,0,1707.245535,44.150364
1,1,2329.605163,60.244947
